In [1]:
pip install SpeechRecognition pyttsx3 openai requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 40.0 MB/s eta 0:00:00


In [8]:
!pip install gTTS playsound


  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=e337f53884ee7bbec2983bc5f43f92172d0c039599afe8ecc4df2638d4446e7c
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [20]:
!pip install gTTS playsound SpeechRecognition openai requests
!pip install pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [22]:
import os
import openai
import requests
from gtts import gTTS
from IPython.display import Audio  # Import Audio from IPython
import sqlite3
import datetime
import speech_recognition as sr
from pydub import AudioSegment  # For MP3 to WAV conversion

# Initialize OpenAI API with your API key
openai.api_key = 'sk-1234abcd1234abcd1234abcd1234abcd1234abcd'  # Replace with your actual OpenAI API key

# Initialize OpenWeatherMap API with your API key
weather_api_key = '5f686dd2df524daf8ff152042250402'  # Your OpenWeatherMap API key

# Create SQLite database for storing tasks
def init_db():
    conn = sqlite3.connect('tasks.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS tasks (task_id INTEGER PRIMARY KEY, task TEXT)''')
    conn.commit()
    conn.close()

# Function to talk to the user using gTTS
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    # Use IPython's Audio display to play the sound
    return Audio("response.mp3", autoplay=True)

# Convert MP3 file to WAV
def convert_mp3_to_wav(mp3_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    wav_file_path = mp3_file_path.replace(".mp3", ".wav")
    audio.export(wav_file_path, format="wav")
    return wav_file_path

# Function to process an uploaded audio file instead of live microphone input
def listen_from_file(file_path):
    recognizer = sr.Recognizer()

    # Convert the uploaded MP3 to WAV for speech recognition if needed
    if file_path.endswith(".mp3"):
        file_path = convert_mp3_to_wav(file_path)

    audio_file = sr.AudioFile(file_path)
    with audio_file as source:
        audio = recognizer.record(source)

    try:
        query = recognizer.recognize_google(audio)
        print(f"User said: {query}")
        return query.lower()
    except Exception as e:
        print("Sorry, I couldn't understand that.")
        return None

# Function to fetch weather information from OpenWeatherMap
def get_weather():
    city = "New York"  # You can make this dynamic later by taking input from the user
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}"
    response = requests.get(url)
    weather_data = response.json()

    # Print the weather data for debugging
    print(weather_data)

    if weather_data["cod"] == "404":
        speak("Sorry, I couldn't find the weather for that city.")
    else:
        main = weather_data["main"]
        weather_desc = weather_data["weather"][0]["description"]
        temperature = main["temp"] - 273.15  # Convert from Kelvin to Celsius
        speak(f"The weather in {city} is {weather_desc} with a temperature of {temperature:.2f} degrees Celsius.")

# Function to get GPT-based response from OpenAI
def get_gpt_response(query):
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",  # You can choose another engine if needed
            prompt=query,
            max_tokens=100
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error in OpenAI request: {e}")
        speak("Sorry, I encountered an error while processing your request.")
        return None

# Task Management functions (add, view, remove tasks)
def add_task(task):
    conn = sqlite3.connect('tasks.db')
    c = conn.cursor()
    c.execute('INSERT INTO tasks (task) VALUES (?)', (task,))
    conn.commit()
    conn.close()

def view_tasks():
    conn = sqlite3.connect('tasks.db')
    c = conn.cursor()
    c.execute('SELECT * FROM tasks')
    tasks = c.fetchall()
    conn.close()
    if tasks:
        speak("Here are your tasks:")
        for task in tasks:
            speak(f"Task {task[0]}: {task[1]}")
    else:
        speak("You have no tasks.")

def remove_task(task_id):
    conn = sqlite3.connect('tasks.db')
    c = conn.cursor()
    c.execute('DELETE FROM tasks WHERE task_id=?', (task_id,))
    conn.commit()
    conn.close()

# Main function to process commands
def main():
    init_db()
    speak("Hello! How can I assist you today?")

    # Use your uploaded audio file (updated with the correct file path)
    uploaded_audio_path = '/content/harvard.wav'  # Update with the correct path to your uploaded audio file

    query = listen_from_file(uploaded_audio_path)

    if query:
        if "weather" in query:
            get_weather()
        elif "tell me a joke" in query:
            speak("Why don't skeletons fight each other? They don't have the guts!")
        elif "time" in query:
            current_time = datetime.datetime.now().strftime("%H:%M")
            speak(f"The current time is {current_time}")
        elif "task" in query:
            if "add" in query:
                task = query.replace("add task", "").strip()
                add_task(task)
                speak(f"Task '{task}' added.")
            elif "view" in query:
                view_tasks()
            elif "remove" in query:
                try:
                    task_id = int(query.split()[-1])
                    remove_task(task_id)
                    speak(f"Task {task_id} removed.")
                except ValueError:
                    speak("Please specify the task number you want to remove.")
        elif "exit" in query or "stop" in query:
            speak("Goodbye! Have a great day!")
        else:
            response = get_gpt_response(query)
            if response:
                speak(response)

if __name__ == "__main__":
    main()


User said: the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and Zest a salt pickle taste fine with ham tacos Al pastor are my favourite a zestful food is the hot cross bun
Error in OpenAI request: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

